In [10]:
import numpy as np
from glob import glob
import pandas as pd
import random
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import auc
import pickle
import pyod
from pyod.utils.stat_models import pairwise_distances_no_broadcast
import warnings
from icecream import ic
from src.constant import translation_dict
warnings.filterwarnings("ignore")


In [2]:
import pandas as pd
import glob

# 获取所有的CSV文件
csv_path = 'data/Train'
pkl_path = 'data/pkl/Train'
csv_files = glob.glob(csv_path+'/*.csv')
pkl_files = glob.glob(pkl_path+'/*.pkl')

In [3]:
with_label_df_list = [pd.read_csv(f) for f in csv_files]
non_label_df_list = [pd.read_csv(f).drop('label', axis=1)  for f in csv_files]


In [5]:
df_list = [pd.read_csv(f) for f in csv_files]

random.shuffle(df_list)
split_point = int(len(df_list) * 0.3)  # 假设你想要80%的数据作为训练集

X_list = [df.drop('label', axis=1) for df in df_list]
y_list = [df['label'] for df in df_list]

train = df_list[:split_point]
test = df_list[split_point:]
ic(len(train), len(test));

In [12]:
ic(X_list[0].info(), y_list[0].info());

ic| X_list[0]

.info(): None, y_list[0].info(): None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volt             256 non-null    float64
 1   current          256 non-null    float64
 2   soc              256 non-null    float64
 3   max_single_volt  256 non-null    float64
 4   min_single_volt  256 non-null    float64
 5   max_temp         256 non-null    float64
 6   min_temp         256 non-null    float64
 7   timestamp        256 non-null    float64
dtypes: float64(8)
memory usage: 16.1 KB
<class 'pandas.core.series.Series'>
RangeIndex: 256 entries, 0 to 255
Series name: label
Non-Null Count  Dtype
--------------  -----
256 non-null    int64
dtypes: int64(1)
memory usage: 2.1 KB


In [ ]:
ic(len(df_list));
ic(df_list[0].shape);

In [27]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self, X_list, y_list):
        self.X = X_list
        self.y = y_list
        
        self.length=len(self.X)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.X[idx].values,self.y[idx].values

dataset = MyDataset(X_list=X_list, y_list=y_list)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [29]:
# 迭代 DataLoader
for batch_idx, (data, target) in enumerate(data_loader):
    print("Batch idx {}, data shape {}, target shape {}".format(
        batch_idx, data.shape, target.shape))
    ic(data.shape, target.shape);
    # 你可以在这里添加你的测试代码，例如检查数据和目标的形状、类型等
    # assert data.shape == (64, 1, 28, 28)
    # assert target.shape == (64,)
    
    # 为了测试，我们只迭代一次
    break

ic| data.shape: torch.Size([64, 256, 8])
    target.shape: torch.Size([64, 256])


Batch idx 0, data shape torch.Size([64, 256, 8]), target shape torch.Size([64, 256])


In [ ]:
dataset.data[0]

# 模型 MLP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(in_features=8*256, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=1)
        self.m = nn.Sigmoid()

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.m(x)
        return x

if __name__ == "__main__":
    net = MyNet()
    print(net)
    input = torch.randn(16, 8, 256)
    out = net(input)
    print(out.size())

In [ ]:
train[1].info()